# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
# Note: pandas and numpy are already imported as pd and np
# The following tables are loaded as pandas DataFrames with the same names: fct_guest_spending
# Please print your final result or dataframe

#convert date to datetime
fct_guest_spending["visit_date"] = pd.to_datetime(fct_guest_spending["visit_date"])

# Filter July 2024 records
july_spending = fct_guest_spending[
	(fct_guest_spending["visit_date"].dt.year == 2024) &
	(fct_guest_spending["visit_date"].dt.month == 7)

]

# Get list of all park experience types (unique)
all_experience_types = fct_guest_spending["park_experience_type"].unique()

# Calculate average spending for July 2024
avg_spending = july_spending.groupby("park_experience_type")["amount_spent"].mean().reset_index()


# Add 0.0 for experience types that have no action taken
avg_spending = avg_spending.set_index("park_experience_type").reindex(all_experience_types, fill_value=0.0).reset_index()

# Print Result
print("Average spending per guest per visit for July 2024:")
print(avg_spending)

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
# convert date to datetime
fct_guest_spending["visit_date"]= pd.to_datetime(fct_guest_spending["visit_date"])

# Filter for August 2024
august_spending = fct_guest_spending[
	(fct_guest_spending["visit_date"].dt.year == 2024 ) &
	(fct_guest_spending["visit_date"].dt.month == 8)
]

# Select guests with more than 1 visit
multi_visitors = august_spending.groupby("guest_id").filter(lambda x: len(x) > 1)

# Find first and last visit expenses
first_last = multi_visitors.sort_values(["guest_id", "visit_date"]).groupby("guest_id").agg(
	first_visit = ("amount_spent", "first"),
	last_visit = ("amount_spent", "last")
).reset_index()

# Add the spending difference
first_last["spending_difference"] = first_last["last_visit"] - first_last["first_visit"]

# Print Result
print("Spending difference between first and last visits in August 2024:")
print(first_last)

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
# Convert date column to datetime
fct_guest_spending["visit_date"] = pd.to_datetime(fct_guest_spending["visit_date"])

# Filter for September 2024
september_spending = fct_guest_spending[
    (fct_guest_spending["visit_date"].dt.year == 2024) &
    (fct_guest_spending["visit_date"].dt.month == 9)
]

# Calculate total spending per guest
guest_totals = september_spending.groupby("guest_id")["amount_spent"].sum().reset_index()

#Exclude guests with zero spending
guest_totals = guest_totals[guest_totals["amount_spent"] > 0]

# Categorize into Low / Medium / Hig
def categorize_spending(amount):
    if amount < 50:
        return "Low"
    elif amount < 100:
        return "Medium"
    else:
        return "High"
			
guest_totals["spending_segment"] = guest_totals["amount_spent"].apply(categorize_spending)


# Print detailed results
print("Guest Spending Segments for September 2024:")
print(guest_totals)
# Optional: summary count of guests in each segment
print("\nSummary of segments:")
print(guest_totals["spending_segment"].value_counts())

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)